In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

In [19]:
def decompose(w):
        return (w+abs(w))/2,(-w+abs(w))/2

## one h net

Supposons un reseau simple : de type one_net d architecture 2 5 1

avec une activation relu : 2 5 (relu) 1 (relu)

W1 = np.array([[2,0,-3,-4,2],[-1,0,2,1,-2]]).reshape(5,2)
W1,W1.shape

In [208]:
W1 = np.array([[-1,1,1,-4,3],[1,-3,2,1,2]]).T
W1


array([[-1,  1],
       [ 1, -3],
       [ 1,  2],
       [-4,  1],
       [ 3,  2]])

b1 = np.array([2,0,-3,-4,2]).reshape(5,1)
b1,b1.shape

In [209]:
b1 = np.array([1,-1,2,0,-2]).reshape(5,1)
b1

array([[ 1],
       [-1],
       [ 2],
       [ 0],
       [-2]])

W2= np.array([-1,-2,0,1,-5]).reshape(1,5)
W2,W2.shape

In [226]:
W2= np.array([1,2,1,-1,-3]).reshape(1,-1)

b2 = np.array([2])
b2,b2.shape

In [227]:
b2 = np.array([0])

x = np.array([1,2])
a = W1.dot(x)+b1
a,W2.dot(a+b2)

----------------------------------

premiere couche


In [228]:
def G1(x):
    return decompose(W1)[1].dot(x)

def H1(x):
    return decompose(W1)[0].dot(x)+b1

def F1(x):
    return np.maximum(G1(x),H1(x))

def NU1(x):
    return F1(x)-G(x)

G1(x),H1(x),F1(x),NU1(x)

In [229]:
PG1 = np.concatenate((decompose(W1)[1],0*b1.reshape(-1,1)),axis=1)
PH1 = np.concatenate((decompose(W1)[0],b1.reshape(-1,1)),axis=1)
PF1 = np.stack((PG1,PH1),axis=1)

In [230]:
PF1,PF1.shape

(array([[[ 1.,  0.,  0.],
         [ 0.,  1.,  1.]],
 
        [[ 0.,  3.,  0.],
         [ 1.,  0., -1.]],
 
        [[ 0.,  0.,  0.],
         [ 1.,  2.,  2.]],
 
        [[ 4.,  0.,  0.],
         [ 0.,  1.,  0.]],
 
        [[ 0.,  0.,  0.],
         [ 3.,  2., -2.]]]), (5, 2, 3))

ATTENTION : ici PG1_i et PH1_i sont des points et PF1_i un segment

________________

deuxieme couche derniere couche, une sortie, c est donc relativement facile


H2_PF1 = np.zeros((5,2,3))
for i in range(5):
        H2_PF1[i]= PF1[i] * decompose(W2)[1][0][i]
        
H2_PG1 = np.zeros((5,2,3))
for i in range(5):
        H2_PG1[i]= PG1[i] * decompose(W2)[0][0][i]
        
G2_PF1 = np.zeros((5,2,3))
for i in range(5):
        G2_PF1[i]= PF1[i] * decompose(W2)[0][0][i]
        
G2_PG1 = np.zeros((5,2,3))
for i in range(5):
        G2_PG1[i]= PG1[i] * decompose(W2)[1][0][i]    
        

In [234]:
H2 = []
for i in range(5):
    if decompose(W2)[1][0][i] !=0:
        H2.append(PF1[i] * decompose(W2)[1][0][i])
        print('f',i+1,decompose(W2)[1][0][i])
    else:
        H2.append(PG1[i] * decompose(W2)[0][0][i])
        print('g',i+1,decompose(W2)[0][0][i])     
        

g 1 1.0
g 2 2.0
g 3 1.0
f 4 1.0
f 5 3.0


In [235]:
G2 = []
for i in range(5):
    if decompose(W2)[1][0][i] !=0:
        G2.append(PG1[i] * decompose(W2)[1][0][i])
        print('g',i+1,decompose(W2)[1][0][i])
    else:
        G2.append(PF1[i] * decompose(W2)[0][0][i])
        print('f',i+1,decompose(W2)[0][0][i])

f 1 1.0
f 2 2.0
f 3 1.0
g 4 1.0
g 5 3.0


In [236]:
H2

[array([1., 0., 0.]),
 array([0., 6., 0.]),
 array([0., 0., 0.]),
 array([[4., 0., 0.],
        [0., 1., 0.]]),
 array([[ 0.,  0.,  0.],
        [ 9.,  6., -6.]])]

In [237]:
G2

[array([[1., 0., 0.],
        [0., 1., 1.]]), array([[ 0.,  6.,  0.],
        [ 2.,  0., -2.]]), array([[0., 0., 0.],
        [1., 2., 2.]]), array([4., 0., 0.]), array([0., 0., 0.])]

In [238]:
for i in H2:
    print(i)
    print()

[1. 0. 0.]

[0. 6. 0.]

[0. 0. 0.]

[[4. 0. 0.]
 [0. 1. 0.]]

[[ 0.  0.  0.]
 [ 9.  6. -6.]]



In [239]:
for i in G2:
    print(i)
    print()

[[1. 0. 0.]
 [0. 1. 1.]]

[[ 0.  6.  0.]
 [ 2.  0. -2.]]

[[0. 0. 0.]
 [1. 2. 2.]]

[4. 0. 0.]

[0. 0. 0.]



On a des points et des lignes

Il faut maintenant faire une somme de Minkowski de tout ca, 

Ensuite prendre convex hull pour trouver f

puis projeter puis passer au squelette !